Course: Computational Statistics
# Topic: Comparison in algorithms for Decision Tree

author: Po-Tse Cheng

potential some shit here

In [ ]:
DGP here generated by the paper

SHIT REGARDING CART

In [ ]:
algorithm: CART

In [ ]:
SOME  OTHER SHIT REGARDING C4.5

In [ ]:
algorithm:c4.5 with J48

In [ ]:
Quest

Benchmark runtime/ accuracy/ also give ratings

In [ ]:
ru

reference